# Multiple GPU in cuGraph

We will be analyzing **1.47 billion social relations** on 41.7 million user profiles from the Twitter dataset.  The CSV file is 26GB and was collected in :<br>
*What is Twitter, a social network or a news media? Haewoon Kwak, Changhyun Lee, Hosung Park, and Sue Moon. 2010.*<br> 

Notice that the memory requirement to read this 26GB dataset is already bigger than the memory of a single GPU. While we are not limited by the device memory size in this case, the whole system should still have at least 60GB of memory available

## PageRank with cuGraph
### Basic setup

In [1]:
# Import needed libraries. We recommend using cugraph_dev env through conda
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import cugraph.comms as Comms
import cugraph.dask as dask_cugraph
import cugraph
import dask_cudf
import time

### Get the data

The Twitter dataset is in our S3 bucket and zipped.  
1. We'll need to create a folder for our data in the `/data` folder
1. Download the zipped data into that folder from S3 (it will take some time as it it 6GB)
1. Decompress the zipped data for use (it will take some time as it it 26GB)

### Initialize multi-GPU environment
Before we get started, we need to setup a Dask local cluster of workers to execute our work and a client to coordinate and schedule work for that cluster. As we see below, we can initiate a cluster and client using only 3 lines of code.

In [2]:
cluster = LocalCUDACluster()
client = Client(cluster)
Comms.initialize(p2p=True)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [3]:
client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 186.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38687,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 186.72 GiB
Comm: tcp://127.0.0.1:40121,Total threads: 1
Dashboard: http://127.0.0.1:36495/status,Memory: 46.68 GiB
Nanny: tcp://127.0.0.1:33961,


### Read the data from disk
cuGraph depends on cudf for data loading and the initial DataFrame creation. The CSV data file contains an edge list, which represents the connection of a vertex to another. The source to destination pairs is what is known as Coordinate Format (COO). In this test case, the data is just two columns. 

In [4]:
input_data_path = '../data/twitter-2010.csv'

# Start ETL timer
t_start = time.time()

# Helper function to set the reader chunk size to automatically get one partition per GPU  
chunksize = dask_cugraph.get_chunksize(input_data_path)

# Multi-GPU CSV reader
e_list = dask_cudf.read_csv(input_data_path, chunksize = chunksize, delimiter=' ', names=['src', 'dst'], dtype=['int32', 'int32'])


### Create a graph


In [5]:
# Create a directed graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
#G = cugraph.Graph(directed=True)
G = cugraph.DiGraph()
G.from_dask_cudf_edgelist(e_list, source='src', destination='dst')

# Print time
print("Read, load and renumber: ", time.time()-t_start, "s")

Read, load and renumber:  5.226404666900635 s


### Call PageRank algorithm


In [6]:
# Start Pagerank timer
t_start = time.time()

# Get the pagerank scores
pr_df = dask_cugraph.pagerank(G, tol=1e-4)

# Print time
print("Pagerank: ", time.time()-t_start, "s")

Pagerank:  119.72036933898926 s


It was that easy! PageRank should only take a few seconds to run on this 26GB input with one GPU.<br>
Check out how it compares to published Spark results in the [Annex](#annex_cell).

### Further analysis on the PageRank result

We can now identify the most influent users in the network.<br>
Notice that the PageRank result is already in a regular `cudf.DataFrame`. We can then sort by PageRank value and print the *Top 3*.

In [7]:
# Sort, descending order
pr_sorted_df = pr_df.sort_values('pagerank',ascending=False)

In [8]:
# Print top 15
print(pr_sorted_df.head(15))

         pagerank    vertex
2624356  0.000368  21513299
15264    0.000292  23933989
7830367  0.000247  23933986
8840     0.000218  21496201
7833295  0.000208  23934048
5223291  0.000174  23937213
2622339  0.000171  23934131
2604623  0.000131  23934073
8771     0.000125  23934123
5230417  0.000082  21515805
7841950  0.000074  23934033
5222330  0.000074  21515803
7827601  0.000073  21515742
5225326  0.000072  21515771
7831688  0.000072  21515862


We can now use the [map](https://s3.us-east-2.amazonaws.com/rapidsai-data/cugraph/benchmark/twitter-2010-ids.csv.gz) to convert Vertex ID into to Twitter's numeric ID. The user name can also be retrieved using the [TwitterID](https://tweeterid.com/) web app.<br>
The table below shows more information on our *Top 3*. Notice that this ranking is much better at capturing network influence compared the number of followers for instance. Further analysis of this dataset was published [here](https://doi.org/10.1145/1772690.1772751).

| Vertex ID	| Twitter ID	| User name	| Description |
| --------- |  ---------   | --------   |   ----------  |
| 21513299	| 813286	| barackobama	| US President (2009-2017) |
| 23933989	| 14224719	| 10DowningStreet | UK Prime Minister office |
| 23933986	| 15131310	| WholeFoods	| Food store from Austin |

